# **6. Scaling**

Here we will:

    - Create the scorecards
    - Generate the points map dictionary
    - Predict the credit score from an input

## **6.1 Create Scorecards**

Assign score to each attribute by specifying:

    - Odds of good of 30:1 at 300 point score and
    - 20 PDO (points to double the odds of good)

Thus, we can calculate the offset and factor:

**- Factor = PDO/ln(2)**

**- Offset = Score - Factor * ln(Odds of good)**

In [1]:
# Import library
import pandas as pd
import numpy as np

# Load configuration
import src.utils as utils

Update the config file to define the references and dump the scorecards.

In [2]:
# Update the config file
config_data = utils.config_load()
config_data

{'raw_dataset_path': 'data/raw/Training Data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'risk_flag',
 'test_size': 0.3,
 'num_columns': ['income',
  'age',
  'experience',
  'current_job_years',
  'current_house_years'],
 'cat_columns': ['married',
  'house_ownership',
  'car_ownership',
  'profession',
  'city',
  'state'],
 'num_of_bins': 4,
 'num_of_cv': 1

In [3]:
# Function to convert the model's output into score points
def scaling():
    """Function to assign score points to each attribute"""
    
    # Define the references: score, odds, pdo
    pdo = config_data['pdo']
    score = config_data['score_ref']
    odds = config_data['odds_ref']

    # Load the best model
    best_model_path = config_data['best_model_path']
    best_model = utils.pickle_load(best_model_path)

    # Load the WOE table
    WOE_table_path = config_data['WOE_table_path']
    WOE_table = utils.pickle_load(WOE_table_path)

    # Load the best model's estimates table
    best_model_summary_path = config_data['best_model_summary_path']
    best_model_summary = utils.pickle_load(best_model_summary_path)

    # Calculate factor and offset
    factor = pdo/np.log(2)
    offset = score-(factor*np.log(odds))

    print('===================================================')
    print(f"Odds of good of {odds}:1 at {score} points score.")
    print(f"{pdo} PDO (points to double the odds of good).")
    print(f"Offset = {offset:.2f}")
    print(f"Factor = {factor:.2f}")
    print('===================================================')

    # Define n = number of characteristics
    n = best_model_summary.shape[0] - 1

    # Define b0
    b0 = best_model.intercept_[0]

    # Adjust characteristic name in best_model_summary_table
    num_cols = config_data['num_columns']
    for col in best_model_summary['Characteristic']:

        if col in num_cols:
            bin_col = col + '_bin'
        else:
            bin_col = col

        best_model_summary.replace(col, bin_col, inplace = True)

    # Merge tables to get beta/parameter estimate for each characteristic
    scorecards = pd.merge(left = WOE_table,
                        right = best_model_summary,
                        how = 'left',
                        on = ['Characteristic'])

    # Define beta and WOE
    beta = scorecards['Estimate']
    WOE = scorecards['WOE']

    # Calculate the score point for each attribute
    scorecards['Points'] = round((offset/n) - factor*((b0/n) + (beta*WOE)))
    scorecards['Points'] = scorecards['Points'].astype('int')

    # Validate
    print('Scorecards table shape : ', scorecards.shape)

    # Dump the scorecards
    scorecards_path = config_data['scorecards_path']
    utils.pickle_dump(scorecards, scorecards_path)

    return scorecards

In [4]:
# Check the function
scaling()

Odds of good of 30:1 at 300 points score.
20 PDO (points to double the odds of good).
Offset = 201.86
Factor = 28.85
Scorecards table shape :  (424, 5)


,Characteristic,Attribute,WOE,Estimate,Points
0,income_bin,"(10309.999, 2511105.0]",-0.045605,-1.048982,17
1,income_bin,"(2511105.0, 5001124.0]",0.042678,-1.048982,20
2,income_bin,"(5001124.0, 7475292.0]",0.042740,-1.048982,20
3,income_bin,"(7475292.0, 9999938.0]",-0.037107,-1.048982,17
4,age_bin,"(20.999, 35.0]",-0.117910,-0.845304,15
...,...,...,...,...,...
419,state,Tripura,-0.409416,0.007332,18
420,state,Uttar_Pradesh,0.044885,0.007332,18
421,state,Uttar_Pradesh[5],-0.090336,0.007332,18
422,state,Uttarakhand,0.607073,0.007332,18


## **6.2 Predict the Credit Score**

Here we need to generate the points map dictionary to predict the credit score from an input.

Update the config file to dump the points map dictionary and the credit score.

In [8]:
# Update the config file
config_data = utils.config_load()
config_data

{'raw_dataset_path': 'data/raw/Training Data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'risk_flag',
 'test_size': 0.3,
 'num_columns': ['income',
  'age',
  'experience',
  'current_job_years',
  'current_house_years'],
 'cat_columns': ['married',
  'house_ownership',
  'car_ownership',
  'profession',
  'city',
  'state'],
 'num_of_bins': 4,
 'num_of_cv': 1

In [9]:
# Generate the points map dict function
def get_points_map_dict():
    """Get the Points mapping dictionary"""
    # Load the scorecards table
    scorecards = utils.pickle_load(config_data['scorecards_path'])

    # Initialize the dictionary
    points_map_dict = {}
    unique_char = set(scorecards['Characteristic'])
    for char in unique_char:
        # Get the Attribute & WOE info for each characterics
        current_data = (scorecards
                            [scorecards['Characteristic']==char]
                            [['Attribute', 'Points']])

        # Get the mapping
        points_map_dict[char] = {}
        for idx in current_data.index:
            attribute = current_data.loc[idx, 'Attribute']
            points = current_data.loc[idx, 'Points']
            points_map_dict[char][attribute] = points

    # Validate data
    print('Number of key : ', len(points_map_dict.keys()))

    # Dump
    utils.pickle_dump(points_map_dict, config_data['points_map_dict_path'])

    return points_map_dict


In [10]:
# Check the function
get_points_map_dict()

Number of key :  11


{'state': {'Andhra_Pradesh': 18,
  'Assam': 18,
  'Bihar': 18,
  'Chandigarh': 18,
  'Chhattisgarh': 18,
  'Delhi': 18,
  'Gujarat': 18,
  'Haryana': 18,
  'Himachal_Pradesh': 18,
  'Jammu_and_Kashmir': 18,
  'Jharkhand': 18,
  'Karnataka': 18,
  'Kerala': 18,
  'Madhya_Pradesh': 18,
  'Maharashtra': 18,
  'Manipur': 18,
  'Mizoram': 18,
  'Odisha': 18,
  'Puducherry': 18,
  'Punjab': 18,
  'Rajasthan': 18,
  'Sikkim': 18,
  'Tamil_Nadu': 18,
  'Telangana': 18,
  'Tripura': 18,
  'Uttar_Pradesh': 18,
  'Uttar_Pradesh[5]': 18,
  'Uttarakhand': 18,
  'West_Bengal': 18},
 'house_ownership': {'norent_noown': 25, 'owned': 28, 'rented': 18},
 'current_job_years_bin': {Interval(-0.001, 3.0, closed='right'): 17,
  Interval(3.0, 6.0, closed='right'): 19,
  Interval(6.0, 9.0, closed='right'): 19,
  Interval(9.0, 14.0, closed='right'): 18},
 'experience_bin': {Interval(-0.001, 5.0, closed='right'): 15,
  Interval(5.0, 10.0, closed='right'): 20,
  Interval(10.0, 15.0, closed='right'): 18,
  Interv

Next, transform the raw input data into score points.

In [11]:
def transform_points(raw_data=None, type=None, config_data=None):
    """Replace data value with points"""
    # Load the numerical columns
    num_cols = config_data['num_columns']

    # Load the points_map_dict
    points_map_dict = utils.pickle_load(config_data['points_map_dict_path'])

    # Load the saved data if type is not None
    if type is not None:
        raw_data = utils.pickle_load(config_data[f'{type}_path'][0])

    # Map the data
    points_data = raw_data.copy()
    for col in points_data.columns:
        if col in num_cols:
            map_col = col + '_bin'
        else:
            map_col = col

        points_data[col] = points_data[col].map(points_map_dict[map_col])

    # Dump data
    if type is not None:
        utils.pickle_dump(points_data, config_data[f'X_{type}_points_path'])

    return points_data

In [12]:
# Check the function on the train set
X_train_points = transform_points(type='train', config_data=config_data)

X_train_points

,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
203209,20,19,21,18,18,21,20,10,18,19,19
42903,17,19,18,18,18,21,16,21,18,18,19
59095,20,19,15,18,18,17,19,17,18,19,19
69516,17,19,15,24,18,17,16,21,18,17,18
232569,20,20,20,24,18,17,18,11,18,19,19
...,...,...,...,...,...,...,...,...,...,...,...
182909,20,20,15,18,18,17,20,29,18,19,19
197799,20,19,15,18,18,17,18,13,18,19,19
228682,17,19,15,18,18,17,16,30,18,19,18
114068,17,19,20,18,18,17,22,38,18,19,18


Then, add a function to calculate the credit score.

Update the config file to dump the credit score.

In [13]:
# Update the config file
config_data = utils.config_load()
config_data

{'raw_dataset_path': 'data/raw/Training Data.csv',
 'dataset_path': 'data/output/data.pkl',
 'predictors_set_path': 'data/output/predictors.pkl',
 'response_set_path': 'data/output/response.pkl',
 'train_path': ['data/output/X_train.pkl', 'data/output/y_train.pkl'],
 'test_path': ['data/output/X_test.pkl', 'data/output/y_test.pkl'],
 'data_train_path': 'data/output/data_train.pkl',
 'data_train_binned_path': 'data/output/data_train_binned.pkl',
 'crosstab_list_path': 'data/output/crosstab_list.pkl',
 'WOE_table_path': 'data/output/WOE_table.pkl',
 'IV_table_path': 'data/output/IV_table.pkl',
 'WOE_map_dict_path': 'data/output/WOE_map_dict.pkl',
 'X_train_woe_path': 'data/output/X_train_woe.pkl',
 'response_variable': 'risk_flag',
 'test_size': 0.3,
 'num_columns': ['income',
  'age',
  'experience',
  'current_job_years',
  'current_house_years'],
 'cat_columns': ['married',
  'house_ownership',
  'car_ownership',
  'profession',
  'city',
  'state'],
 'num_of_bins': 4,
 'num_of_cv': 1

In [14]:
# Function to predict the credit score
def predict_score(raw_data, config_data):
    """Function to predict the credit score"""

    points = transform_points(raw_data = raw_data,
                            type = None,
                            config_data = config_data)

    score = int(points.sum(axis=1))

    utils.pickle_dump(score, config_data['score_path'])

    return score

In [15]:
# Check the function with raw data input
tes_input = {
    'income_bin': 4000000,
    'age_bin': 25,
    'experience_bin': 3,
    'current_job_years_bin': 3,
    'current_house_years_bin': 20,
    'married': 'single',
    'house_ownership': 'owned',
    'car_ownership': 'yes',
    'profession': 'Financial_Analyst',
    'city': 'Orai',
    'state': 'Uttarakhand'
}

tes = pd.DataFrame(tes_input, index=[0])

tes

,income_bin,age_bin,experience_bin,current_job_years_bin,current_house_years_bin,married,house_ownership,car_ownership,profession,city,state
0,4000000,25,3,3,20,single,owned,yes,Financial_Analyst,Orai,Uttarakhand


In [16]:
# Predict the credit score
predict_score(raw_data=tes, config_data=config_data)

/tmp/ipykernel_695/514670272.py:9: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  score = int(points.sum(axis=1))


216